### Imports

In [1]:
import torch
import torch.nn as nn
import torch_directml
import torchmetrics
import torch.optim as optim
import torchmetrics.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')
#AMD GPU Acceleration
device = torch_directml.device(torch_directml.default_device())

C:\Users\cafes\Envs\torch-dml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed()

In [4]:
train_writer = SummaryWriter('.\\runs\\train\\rnn\\')
val_writer = SummaryWriter('.\\runs\\validate\\rnn\\')

### Choose One

#### Read in Standardized Downsample with new feature

In [5]:
train_df = pd.read_csv('train.csv', index_col='Time')
test_df = pd.read_csv('test.csv', index_col='Time')
train_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit,Event
Time,,,,,,,,,
0,-0.416921,0.680190,-0.585780,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
1,-0.420816,0.678475,-0.578919,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
2,-0.419886,0.676579,-0.580286,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
3,-0.413896,0.674399,-0.594393,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
4,-0.403594,0.674941,-0.607099,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
...,...,...,...,...,...,...,...,...,...
136,-0.230050,0.202884,-1.195147,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
137,-0.219754,0.198519,-1.206719,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
138,-0.210724,0.204065,-1.211322,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0


In [6]:
test_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit
Time,,,,,,,,
0,-9.541841,0.572790,-1.409475,0.86653,-0.110647,-1.719166,-0.182777,0.648921
1,-9.544045,0.570599,-1.436634,0.86653,-0.110647,-1.719166,-0.182777,0.648921
2,-9.537242,0.568221,-1.425318,0.86653,-0.110647,-1.719166,-0.182777,0.648921
3,-9.539138,0.570690,-1.411444,0.86653,-0.110647,-1.719166,-0.182777,0.648921
4,-9.548736,0.568310,-1.425458,0.86653,-0.110647,-1.719166,-0.182777,0.648921
...,...,...,...,...,...,...,...,...
281683,-0.896122,-0.228349,0.397413,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281684,-0.898800,-0.228733,0.398382,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281685,-0.898516,-0.227567,0.400168,0.86653,1.272005,-1.719166,-0.342362,-0.792988


#### Read in Subset

In [ ]:
df = pd.read_csv('subset.csv')
test_df = pd.read_csv('test_subset.csv')

#### Read in Downsample

In [6]:
df = pd.read_csv('downsample30.csv')
test_df = pd.read_csv('test_downsample.csv')

In [7]:
train_df = df.drop(columns=['Subject', 'Id']).set_index('Time')
test_df = test_df.drop(columns=['Subject', 'Id']).set_index('Time')
train_df

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
Time,,,,,,,,,,
0,-0.420712,0.673434,-0.591752,3,2.0,1,68,0,9.0,0
1,-0.424601,0.671724,-0.584907,3,2.0,1,68,0,9.0,0
2,-0.423672,0.669832,-0.586271,3,2.0,1,68,0,9.0,0
3,-0.417690,0.667659,-0.600345,3,2.0,1,68,0,9.0,0
4,-0.407401,0.668199,-0.613021,3,2.0,1,68,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...
136,-0.234080,0.197408,-1.199697,20,1.0,0,65,1,22.0,0
137,-0.223797,0.193055,-1.211242,20,1.0,0,65,1,22.0,0
138,-0.214779,0.198586,-1.215835,20,1.0,0,65,1,22.0,0


In [8]:
test_df

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx
Time,,,,,,,,,
0,-9.533939,0.566322,-1.413525,3,2.0,1,68,0,9.0
1,-9.536140,0.564137,-1.440621,3,2.0,1,68,0,9.0
2,-9.529345,0.561765,-1.429331,3,2.0,1,68,0,9.0
3,-9.531239,0.564227,-1.415490,3,2.0,1,68,0,9.0
4,-9.540825,0.561854,-1.429471,3,2.0,1,68,0,9.0
...,...,...,...,...,...,...,...,...,...
281683,-0.899299,-0.232668,0.389147,2,0.0,1,79,0,8.0
281684,-0.901973,-0.233051,0.390114,2,0.0,1,79,0,8.0
281685,-0.901690,-0.231888,0.391896,2,0.0,1,79,0,8.0


### Dataset/DataLoader

In [7]:
class ParkinsonDataset(Dataset):
    def __init__(self, df, lookback=5, goal='train'):
        super().__init__()
        self.data = df.to_numpy()
        self.lookback = lookback
        self.goal = goal

    # Return length of sample
    def __len__(self):
        return self.data.shape[0]

    # Get data at idx
    def __getitem__(self, idx):
        if self.goal == 'train':
            if idx < self.lookback:
                #If the index is less than lookback, pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1]-1, dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)

            features = features.to(device)
            label = label.to(device)
            return features, label

        else:
            if idx < self.lookback:
                #If the index is less than lookback, pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1], dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :], dtype=torch.float32)

            features = features.to(device)
            return features

In [8]:
def train_val_split(df):
    #Split the training dataset into training/validation sets
    #80% Training - 20% Validation
    train_size = int(.8 * len(df))
    val_size = len(df)-train_size
    train_data, val_data = torch.utils.data.random_split(df, [train_size, val_size])

    #Instantiate DataLoaders
    train_loader = DataLoader(train_data,
                              batch_size=1024,
                              shuffle=False)

    val_loader = DataLoader(val_data,
                            batch_size=1024,
                            shuffle=False)
    
    return train_loader, val_loader

In [9]:
train = ParkinsonDataset(train_df, goal='train')
train_loader, val_loader = train_val_split(train)

In [ ]:
test = test_df.drop(columns='Id')
test = ParkinsonDataset(test, goal='test')
test_loader = DataLoader(test, 
                         batch_size=1024, 
                         shuffle=False)

In [10]:
metric = torchmetrics.AveragePrecision(task='multiclass', num_classes=4, average='weighted', thresholds=4).to(device)

### Overfit Batch Function

In [11]:
#Function to determine whether the model is training appropriately
def overfit_batch(model, criterion, optimizer, batch, epochs):
    model = model.to(device)
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        for _ in range(len(batch)):
            features, label = batch
            features = features.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            output = model(features)
            loss = criterion(logits, label.long())
            ypred = torch.softmax(logits, dim=1)
            metric(ypred, label.long())
            precision=metric.compute()
            
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
            precision_total += precision.item()

        avg_loss = loss_total / len(batch)
        avg_precision = precision_total / len(batch)
        if epoch %10 == 9:
            print(f'Epoch [{epoch + 1}/{epochs}] | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}')

### Train | Validate | Predict

In [12]:
def predict(model, dataloader):
    predictions = []
    with tqdm(dataloader, colour='yellow') as pbar:
        for features in pbar:
            features = features.to(device)
            preds = model(features)
            preds = torch.argmax(preds, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

In [13]:
#Validate one epoch
def validate(model, criterion, dataloader, epoch):
    model.eval()
    metric.reset()
    
    loss_total = 0.0
    precision_total = 0.0
    
    with tqdm(dataloader, desc='Validate', total=len(dataloader), unit='batch', colour='blue') as pbar:
        with torch.no_grad():
            for batch in pbar:
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                logits = model(features)               
                #Update running loss and precision
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                precision_total += precision.item()
                loss_total += loss.item()

                #Update the tqdm progress bar description with loss and precision
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')
                
        avg_loss = loss_total / len(dataloader)
        avg_precision = precision_total / len(dataloader)
        
    model.train()    
    return avg_loss, avg_precision

In [14]:
def train_validate(model, criterion, optimizer, scheduler, epochs=100, train_loader=train_loader, val_loader=val_loader, patience=5):
    model = model.train()
    best_precision = None
    min_delta = 0
    counter = 0
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        with tqdm(train_loader, desc='Train', total=len(train_loader), unit='batch', colour='green') as pbar:
            for batch in pbar:
                #Set features/targets to device
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                #Predict labels
                optimizer.zero_grad()                
                logits = model(features)
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                #Tensorboard
                train_writer.add_scalar('Train-Loss', loss, epoch)
                train_writer.add_scalar('Train-Precision', precision, epoch)
                
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')

                #Adjust gradient
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                #Accumulate total loss and precision
                loss_total += loss.item()
                precision_total += precision.item()

            avg_loss = loss_total / len(train_loader)
            avg_precision = precision_total / len(train_loader)
         
        #Report training loss/precision
        print(f'Epoch {epoch + 1} | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}%')

        if epoch%3==2:
            #Validate one epoch and update lr scheduler
            avg_val_loss, avg_val_precision = validate(model, criterion, val_loader, epoch)
            scheduler.step(avg_val_precision)
        
            #Early Stopping
            if best_precision == None:
                best_precision = avg_val_precision
            elif(best_precision - avg_val_precision < min_delta):
                best_precision = avg_val_precision
                counter = 0
            elif(best_precision - avg_val_precision > min_delta):
                counter += 1
                if(counter >= patience):
                    print(f'Early Stopping: Precision stagnated or decreased too many times. Best Validation: {(best_precision*100.0):.2f}%')
                    break

            print(f'Epoch {epoch + 1} | Avg Loss: {avg_val_loss:.4f} | Avg Precision: {(avg_val_precision * 100.0):.2f}% | Patience: {counter}')

    train_writer.flush()
    train_writer.close()
    val_writer.flush()
    val_writer.close()
    print(f'Training Complete - Best Validate Precision: {(best_precision*100.0):.2f}%')
    
    return model

### Model Architecture

In [15]:
class Rnn(nn.Module):
    def __init__(self, num_inputs, hidden_size, num_layers, num_outputs):
        super().__init__()
        self.hidden = hidden_size
        self.layers = num_layers
        self.rnn = nn.RNN(num_inputs,
                            hidden_size,
                            num_layers,
                            batch_first=True)

        # Initialize weights with He initialization
        for name, param in self.rnn.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)
                
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_size, num_outputs)
        nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        h0 = torch.zeros((self.layers, x.size(0), self.hidden), dtype=torch.float32).to(device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]

        # Apply batch normalization
        out = self.batch_norm1(out)
        out = self.elu(out)
        out = self.fc1(out)  
        return out

### Overfit

In [22]:
#overfit batch
batch = next(iter(train_loader))
model = Rnn(3, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr = .01)
epochs=100
overfit_batch(model, criterion, optimizer, batch, epochs)

Epoch [10/100] | Avg Loss: 0.5415 | Avg Accuracy: 78.56
Epoch [20/100] | Avg Loss: 0.3246 | Avg Accuracy: 88.38
Epoch [30/100] | Avg Loss: 0.1651 | Avg Accuracy: 95.02
Epoch [40/100] | Avg Loss: 0.0606 | Avg Accuracy: 98.93
Epoch [50/100] | Avg Loss: 0.0204 | Avg Accuracy: 100.00
Epoch [60/100] | Avg Loss: 0.0073 | Avg Accuracy: 100.00
Epoch [70/100] | Avg Loss: 0.0036 | Avg Accuracy: 100.00
Epoch [80/100] | Avg Loss: 0.0022 | Avg Accuracy: 100.00
Epoch [90/100] | Avg Loss: 0.0015 | Avg Accuracy: 100.00
Epoch [100/100] | Avg Loss: 0.0012 | Avg Accuracy: 100.00


# Training

## Subset

## Downsample

### 10-30-23

#### Model 2 - Lookback 3

In [17]:
model = Rnn(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model2 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model2.state_dict(), '.\\models\\downsample\\rnn\\model2.pth')

Train: 100%|████████████████████████| 295/295 [00:45<00:00,  6.52batch/s, Loss: 0.3310 | Precision: 69.57%]


Epoch 1 | Avg Loss: 0.5954 | Avg Precision: 68.03%


Train: 100%|████████████████████████| 295/295 [00:45<00:00,  6.42batch/s, Loss: 0.4191 | Precision: 69.54%]


Epoch 2 | Avg Loss: 0.5852 | Avg Precision: 69.24%


Train: 100%|████████████████████████| 295/295 [00:45<00:00,  6.49batch/s, Loss: 0.3956 | Precision: 69.87%]


...

Epoch 26 | Avg Loss: 0.5564 | Avg Precision: 70.75%


Train: 100%|████████████████████████| 295/295 [00:48<00:00,  6.09batch/s, Loss: 0.3608 | Precision: 70.93%]


Epoch 27 | Avg Loss: 0.5553 | Avg Precision: 70.83%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.35batch/s, Loss: 0.5727 | Precision: 70.99%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 71.87%
Training Complete


### 10-31-23

#### Model 3 - batch 512

In [17]:
model = Rnn(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model3 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model3.state_dict(), '.\\models\\downsample\\rnn\\model3.pth')

Train: 100%|████████████████████████| 589/589 [00:50<00:00, 11.67batch/s, Loss: 0.3861 | Precision: 69.51%]


Epoch 1 | Avg Loss: 0.5992 | Avg Precision: 68.41%


Train: 100%|████████████████████████| 589/589 [00:50<00:00, 11.78batch/s, Loss: 0.4244 | Precision: 70.22%]


Epoch 2 | Avg Loss: 0.5775 | Avg Precision: 69.96%


Train: 100%|████████████████████████| 589/589 [00:49<00:00, 11.84batch/s, Loss: 0.4698 | Precision: 70.40%]


...

Epoch 99 | Avg Loss: 0.5370 | Avg Precision: 70.98%


Validate: 100%|█████████████████████| 148/148 [00:11<00:00, 12.56batch/s, Loss: 0.1739 | Precision: 71.18%]


Epoch 99 | Avg Loss: 0.5320 | Avg Precision: 71.25% | Patience: 3


Train: 100%|████████████████████████| 589/589 [00:47<00:00, 12.27batch/s, Loss: 0.3976 | Precision: 71.07%]

Epoch 100 | Avg Loss: 0.5362 | Avg Precision: 70.93%
Training Complete


### 11-1-23

#### Model 4 - Batch 250 - Lookback 2

In [18]:
model = Rnn(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2)
epochs = 100

model4 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model4.state_dict(), '.\\models\\downsample\\rnn\\model4.pth')

Train: 100%|██████████████████████| 1205/1205 [01:03<00:00, 18.87batch/s, Loss: 0.5095 | Precision: 67.83%]


Epoch 1 | Avg Loss: 0.6283 | Avg Precision: 67.18%


Train: 100%|██████████████████████| 1205/1205 [01:04<00:00, 18.82batch/s, Loss: 0.4816 | Precision: 68.88%]


Epoch 2 | Avg Loss: 0.6132 | Avg Precision: 68.72%


Train: 100%|██████████████████████| 1205/1205 [01:03<00:00, 19.02batch/s, Loss: 0.4754 | Precision: 69.16%]


...

Epoch 23 | Avg Loss: 0.5769 | Avg Precision: 69.57%


Train: 100%|██████████████████████| 1205/1205 [01:04<00:00, 18.77batch/s, Loss: 0.4449 | Precision: 69.77%]


Epoch 24 | Avg Loss: 0.5770 | Avg Precision: 69.58%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.60batch/s, Loss: 0.5902 | Precision: 69.79%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 70.01%
Training Complete


### 11-14-23

#### Model 5

In [16]:
model = Rnn(8, 500, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=0, min_lr=.0001, verbose=True)

model5 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model5.state_dict(), '.\\models\\downsample\\rnn\\model5.pth')

Train: 100%|████████████████████████| 295/295 [00:54<00:00,  5.43batch/s, Loss: 0.4432 | Precision: 78.18%]


Epoch 1 | Avg Loss: 0.4573 | Avg Precision: 75.67%


Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.35batch/s, Loss: 0.2301 | Precision: 80.26%]


Epoch 2 | Avg Loss: 0.4000 | Avg Precision: 79.90%


Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.32batch/s, Loss: 0.3118 | Precision: 80.52%]


...

Epoch 99 | Avg Loss: 0.3223 | Avg Precision: 84.33%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.58batch/s, Loss: 0.3234 | Precision: 84.10%]


Epoch 99 | Avg Loss: 0.3271 | Avg Precision: 83.99% | Patience: 0


Train: 100%|████████████████████████| 295/295 [00:53<00:00,  5.48batch/s, Loss: 0.1190 | Precision: 84.47%]

Epoch 100 | Avg Loss: 0.3217 | Avg Precision: 84.32%
Training Complete - Best Validate Precision: 83.99%


## Predict | Submit

In [ ]:
ypreds = predict(model1, test_loader)
test_df['Prediction'] = ypreds

In [ ]:
sub = pd.DataFrame()
temp = test_df.copy()
temp["Id"] = temp.apply(lambda x : str(x.Id) + "_" + str(x.Time), axis = 1)
temp['StartHesitation'] = np.where(temp['Prediction'] == 1, 1, 0)
temp['Turn'] = np.where(temp['Prediction'] == 2, 1, 0)
temp['Walking'] = np.where(temp['Prediction'] == 3, 1, 0)
temp = temp[['Id', 'StartHesitation', 'Turn', 'Walking']]
sub = pd.concat([sub, temp])